# Tutorial 2. Controlling a Transmock setup

This tutorial covers a "real" usage example using the Transmock. We will go over all the aspects relevant in controlling an experiment using the mock transmon. 

The steps we will cover are 
1. Initializing the setup
2. The device and qubit objects 
3. Running basic measurements 
4. Calibrating your setup 



<br>
<div class="alert alert-info">
<b>NOTE:</b> We recommend using PycQED from a console for actual use. 
</div>

# 1. Initializing the setup 

Experiments in `PycQED` are run by starting an iPython kernel (console or notebook) in which we instantiate different instruments that we then interact with. 
A session in one of these kernels typically lasts multiple days/weeks in the case of extended experiments. Before we can start runnning an experiment we start by running an initialization script. Such a script consists several steps. 

1. Importing the required modules. 
2. Setting the datadirectory
3. Instantiating the instruments and (optionally) loading settings onto these instruments

Normally the environment would be instantiated by importing from an external init script e.g.: `from my_init import *`. Here we explicitly put all the parts of the initialization script required to setup a 2 qubit mock experiment. Note that all the instruments being used are mock instruments. 

## importing the required modules

In [1]:
###############################################################################
# Import Statements
###############################################################################

# Generic python imports 
import os
import warnings
import openql
import datetime
import time
import pycqed as pq
import networkx as nx

import matplotlib.pyplot as plt
import numpy as np
from importlib import reload



# generic PycQED/QCoDeS imports 
from qcodes import station
from pycqed.measurement import measurement_control

from pycqed.analysis_v2 import measurement_analysis as ma2
from pycqed.analysis import measurement_analysis as ma

from pycqed.utilities import general as gen
import pycqed.analysis.analysis_toolbox as a_tools

# Package for dependency graph based calibrations
from autodepgraph import AutoDepGraph_DAG


# Annoying warning:
os.environ['PYGSTI_BACKCOMPAT_WARNING'] = '0' # suppresses a warning in PyGSTi 

# Import instruments 
from pycqed.instrument_drivers.meta_instrument.qubit_objects import mock_CCL_Transmon as mct
from pycqed.instrument_drivers.meta_instrument.qubit_objects.qubit_object import Qubit
from pycqed.instrument_drivers.meta_instrument.qubit_objects.CCL_Transmon import CCLight_Transmon
from pycqed.instrument_drivers.meta_instrument.LutMans.ro_lutman import UHFQC_RO_LutMan
from pycqed.instrument_drivers.physical_instruments.QuTech_VSM_Module import Dummy_QuTechVSMModule
from pycqed.instrument_drivers.physical_instruments.QuTech_CCL import dummy_CCL
from pycqed.instrument_drivers.meta_instrument.qubit_objects.CC_transmon import CBox_v3_driven_transmon, QWG_driven_transmon
from pycqed.instrument_drivers.meta_instrument.qubit_objects.Tektronix_driven_transmon import Tektronix_driven_transmon
from pycqed.instrument_drivers.meta_instrument.qubit_objects.QuDev_transmon import QuDev_transmon


from pycqed.instrument_drivers.physical_instruments.QuTech_Duplexer import Dummy_Duplexer
import pycqed.instrument_drivers.physical_instruments.ZurichInstruments.UHFQuantumController as uhf
# from pycqed.instrument_drivers.physical_instruments.QuTech_SPI_S4g_FluxCurrent \
#     import QuTech_SPI_S4g_FluxCurrent
from pycqed.instrument_drivers.meta_instrument.LutMans import mw_lutman as mwl
import pycqed.instrument_drivers.virtual_instruments.virtual_MW_source as vmw
import pycqed.instrument_drivers.virtual_instruments.virtual_SignalHound as sh
import pycqed.instrument_drivers.physical_instruments.ZurichInstruments.ZI_HDAWG8 as HDAWG
import pycqed.instrument_drivers.virtual_instruments.virtual_SPI_S4g_FluxCurrent as flx
import pycqed.instrument_drivers.virtual_instruments.virtual_VNA as VNA
import pycqed.instrument_drivers.meta_instrument.device_dependency_graphs as DDG
import pycqed.instrument_drivers.meta_instrument.device_object_CCL as do
from pycqed.instrument_drivers.meta_instrument.Resonator import resonator







c:\users\niels\documents\github\pycqed_py3\data
Data directory set to: c:\users\niels\documents\github\pycqed_py3\data
c:\users\niels\documents\github\pycqed_py3\data
c:\users\niels\documents\github\pycqed_py3\data
Data directory set to: c:\users\niels\documents\github\pycqed_py3\data
c:\users\niels\documents\github\pycqed_py3\data
Data directory set to: c:\users\niels\documents\github\pycqed_py3\data
c:\users\niels\documents\github\pycqed_py3\data
Data directory set to: c:\users\niels\documents\github\pycqed_py3\data
c:\users\niels\documents\github\pycqed_py3\data
Data directory set to: c:\users\niels\documents\github\pycqed_py3\data


## Setting the datadirectory

In [2]:

test_datadir = os.path.join(pq.__path__[0], 'tests', 'test_output') # we use a test datadirectory for our examples
a_tools.datadir = test_datadir


timestamp = None # '20190719_164604'  
# the timestamp variable is used below to load settings from previous experiments onto instruments

## Instantiating the instruments

In [3]:
###############################################################################
# MC and monitor
###############################################################################
station = station.Station()
# The measurement control is used to control experiments (see tutorial 1.)
MC = measurement_control.MeasurementControl(
    'MC', live_plot_enabled=True, verbose=True)
MC.station = station
station.add_component(MC)
MC.create_plot_monitor()
MC.live_plot_enabled(True)

# Required to set it to the testing datadir
MC.datadir(a_tools.datadir)


###############################################################################
# nested MC
###############################################################################
nested_MC = measurement_control.MeasurementControl(
    'nested_MC', live_plot_enabled=True, verbose=True)
nested_MC.station = station
station.add_component(nested_MC)
nested_MC.datadir(a_tools.datadir)

In [4]:
###############################################################################
# Instruments
###############################################################################
# Fluxcurrent
fluxcurrent = flx.virtual_SPI_S4g_FluxCurrent(
        'fluxcurrent',
        channel_map={
            'FBL_Q1': (0, 0),
            'FBL_Q2': (0, 1),
        })
fluxcurrent.FBL_Q1(0)
fluxcurrent.FBL_Q2(0)
station.add_component(fluxcurrent)

###############################################################################
# VNA
VNA = VNA.virtual_ZNB20('VNA')
station.add_component(VNA)

###############################################################################
# MW sources
MW1 = vmw.VirtualMWsource('MW1')
MW2 = vmw.VirtualMWsource('MW2')
MW3 = vmw.VirtualMWsource('MW3')

###############################################################################
# SignalHound
SH = sh.virtual_SignalHound_USB_SA124B('SH')

###############################################################################
# UHFQC
UHFQC = uhf.UHFQC(name='UHFQC', server='emulator',
                               device='dev2109', interface='1GbE')

###############################################################################
# CCL
CCL = dummy_CCL('CCL')

###############################################################################
# VSM
VSM = Dummy_QuTechVSMModule('VSM')

###############################################################################
# AWG
AWG = HDAWG.ZI_HDAWG8(name='DummyAWG8', server='emulator', num_codewords=32, device='dev8026', interface='1GbE')


AWG8_VSM_MW_LutMan = mwl.AWG8_VSM_MW_LutMan('MW_LutMan_VSM')
AWG8_VSM_MW_LutMan.AWG(AWG.name)
AWG8_VSM_MW_LutMan.channel_GI(1)
AWG8_VSM_MW_LutMan.channel_GQ(2)
AWG8_VSM_MW_LutMan.channel_DI(3)
AWG8_VSM_MW_LutMan.channel_DQ(4)
AWG8_VSM_MW_LutMan.mw_modulation(100e6)
AWG8_VSM_MW_LutMan.sampling_rate(2.4e9)

###############################################################################
# RO Lutman
ro_lutman = UHFQC_RO_LutMan(
    'RO_lutman', num_res=5, feedline_number=0)
ro_lutman.AWG(UHFQC.name)

###############################################################################
# Qubit
Q1 = mct.Mock_CCLight_Transmon('Q1')

# Assign instruments
Q1.instr_LutMan_MW(AWG8_VSM_MW_LutMan.name)
Q1.instr_LO_ro(MW1.name)
Q1.instr_LO_mw(MW2.name)
Q1.instr_spec_source(MW3.name)

Q1.instr_acquisition(UHFQC.name)
Q1.instr_VSM(VSM.name)
Q1.instr_CC(CCL.name)
Q1.instr_LutMan_RO(ro_lutman.name)
Q1.instr_MC(MC.name)
Q1.instr_nested_MC(nested_MC.name)
Q1.instr_FluxCtrl(fluxcurrent.name)
Q1.instr_SH(SH.name)
Q1.cfg_with_vsm(False)
Q1.done_spectroscopy = False

config_fn = os.path.join(
    pq.__path__[0], 'tests', 'openql', 'test_cfg_CCL.json')
Q1.cfg_openql_platform_fn(config_fn)
# QL.dep_graph()
station.add_component(Q1)
# Does not set any initial parameters, it should work from scratch
# Qubit
Q2 = mct.Mock_CCLight_Transmon('Q2')
Q2_parameters = {'mock_Ec': 243e6,
                 'mock_Ej1': 8.348e9,
                 'mock_Ej2': 8.246e9,
                 'mock_fl_dc_I_per_phi0': {'FBL_Q1': 2, 'FBL_Q2': 20.3153e-3},
                 # 'mock_fl_dc_V0'
                 'mock_fl_dc_ch': 'FBL_Q2',
                 'mock_freq_res_bare': 7.35e9,
                 'mock_freq_test_res': 7.73e9,
                 'mock_sweetspot_phi_over_phi0': 0,
                 'mock_Qe': 19000,
                 'mock_Q': 15000,
                 'mock_slope': 0}


for parameter, value in Q2_parameters.items():
    Q2.parameters[parameter](value)
# Assign instruments
Q2.instr_LutMan_MW(AWG8_VSM_MW_LutMan.name)
Q2.instr_LO_ro(MW1.name)
Q2.instr_LO_mw(MW2.name)
Q2.instr_spec_source(MW3.name)

Q2.instr_acquisition(UHFQC.name)
Q2.instr_VSM(VSM.name)
Q2.instr_CC(CCL.name)
Q2.instr_LutMan_RO(ro_lutman.name)
Q2.instr_MC(MC.name)
Q2.instr_nested_MC(nested_MC.name)
Q2.instr_FluxCtrl(fluxcurrent.name)
Q2.instr_SH(SH.name)


config_fn = os.path.join(
    pq.__path__[0], 'tests', 'openql', 'test_cfg_CCL.json')
Q2.cfg_openql_platform_fn(config_fn)
# QR.dep_graph()
station.add_component(Q2)

fakequbit = mct.Mock_CCLight_Transmon('fakequbit')

# Assign instruments
fakequbit.instr_LutMan_MW(AWG8_VSM_MW_LutMan.name)
fakequbit.instr_LO_ro(MW1.name)
fakequbit.instr_LO_mw(MW2.name)
fakequbit.instr_spec_source(MW3.name)

fakequbit.instr_acquisition(UHFQC.name)
fakequbit.instr_VSM(VSM.name)
fakequbit.instr_CC(CCL.name)
fakequbit.instr_LutMan_RO(ro_lutman.name)
fakequbit.instr_MC(MC.name)
fakequbit.instr_nested_MC(nested_MC.name)
fakequbit.instr_FluxCtrl(fluxcurrent.name)
fakequbit.instr_SH(SH.name)
fakequbit.cfg_with_vsm(False)

config_fn = os.path.join(
    pq.__path__[0], 'tests', 'openql', 'test_cfg_CCL.json')
fakequbit.cfg_openql_platform_fn(config_fn)
# fakequbit.dep_graph()
station.add_component(fakequbit)
##############################################################################
# Device
Mock_Octobox = do.DeviceCCL(name='Mock_Octobox')
Mock_Octobox.qubits(['Q1', 'Q2', 'fakequbit'])

Q1.instr_device(Mock_Octobox.name)
Q2.instr_device(Mock_Octobox.name)
fakequbit.instr_device(Mock_Octobox.name)
resQ1 = resonator('2', freq=7.5e9)
resQ2 = resonator('1', freq=7.35e9)
rest1 = resonator('t1', freq=7.73e9, type='test_resonator')
rest2 = resonator('t2', freq=7.8e9, type='test_resonator')

# Mock_Octobox.expected_resonators = [resQR, resQL, rest1, rest2]
###############################################################################
# DepGraph
Qubits = [Q1, Q2, fakequbit]
# some_file.py
#dag = DDG.octobox_dep_graph(name='Octobox', device=Mock_Octobox)
# dag.create_dep_graph(Qubits)
# dag.set_all_node_states('needs calibration')
# dag.set_node_state('QL Drive Mixer Calibrations', 'good')
# dag.set_node_state('QR Drive Mixer Calibrations', 'good')
# dag.set_node_state('QL Readout Mixer Calibrations', 'good')
# dag.set_node_state('QR Readout Mixer Calibrations', 'good')
# ###############################################################################
# # Hacky stuff to make life easier

# # Room temp:
Q1.freq_qubit(5.85e9)
Q2.freq_qubit(5.48e9)

for Q in Qubits:
    # Q.ro_acq_averages(32768*4)
    Q.ro_freq(7.5e9)

# # QL.freq_res(QL.mock_freq_res())
# # QL.ro_pulse_amp_CW(QL.mock_ro_pulse_amp_CW())

# # gen.load_settings_onto_instrument_v2(QL, timestamp=timestamp)
# # gen.load_settings_onto_instrument_v2(fakequbit, timestamp=timestamp)
# dag.maintain_QL_Frequency_at_Sweetspot()
# dag.maintain_QR_Frequency_Fine()
# freqs = np.arange(7.348e9, 7.352e9, 0.025e6)
# powers = np.arange(-40, 1, 1)
# fakequbit.measure_resonator_power(freqs=freqs, powers=powers)


# for power in np.arange(-35, -14, 1):
#     QL.spec_pow(power)
#     QL.find_frequency(freqs=np.arange(5.85e9 - 20e6, 5.85e9+20e6, 0.1e6))

Connected to: None MW1 (serial:None, firmware:None) in 0.12s
Connected to: None MW2 (serial:None, firmware:None) in 0.00s
Connected to: None MW3 (serial:None, firmware:None) in 0.00s
Initialized SignalHound in 0.00s
Setting debug level to 0
Connected to: ZurichInstruments UHFQA (serial:dev2109, firmware:99999) in 1.77s
Connected to: None None (serial:None, firmware:None) in 0.01s
Connected to:  VSM (serial:Dummy, firmware:) in 0.10s
Setting debug level to 0
Connected to: ZurichInstruments HDAWG8 (serial:dev8026, firmware:99999) in 1.26s
Connected to: <Mock_CCLight_Transmon: Q1> in 0.01 s
Connected to: <Mock_CCLight_Transmon: Q2> in 0.01 s
Connected to: <Mock_CCLight_Transmon: fakequbit> in 0.01 s


# 2. Running an experiment on a mock transmon

In [5]:
Q1.ro_freq(6e9)
Q1.mw_freq_mod(100e6)
Q1.freq_res(6e9)
Q1.freq_res()
#Q2.fl_dc_ch(fluxcurrent.FBL_Q1())
#Q1.fl_dc_ch(fluxcurrent.FBL_Q2())
Q1.mock_freq_res_bare(7.58726e9)
Q1.mock_sweetspot_phi_over_phi0(0.1)
freq_res = Q1.calculate_mock_resonator_frequency()
Q1.freq_res(7.587e9)

In [7]:
Q1.fl_dc_ch('FBL_Q2')
Q1.freq_qubit(6e9)

In [8]:
#Q1.find_resonator_frequency()
Q2.measure_allxy()

File successfully uploaded in 0.00s


Starting measurement: AllXY_Q2
Sweep function: OpenQL_Sweep
Detector function: raw_UHFQC_integrated_average
[DEBUG UHF SWF] SHOULD HAVE HAD AN ERROR
 100% completed 	elapsed time: 2.4s 	time left: 0.0s


1.3459917282193128

In [10]:
Q1.freq_qubit(6e9)
Q1.find_frequency(freqs=np.arange(6e9,7e9,10e6))

TypeError: 'numpy.complex128' object is not iterable

In [11]:
Q1.measure_T1(times=np.arange(1e-6,60e-6,1e-6))

TypeError: 'numpy.complex128' object is not iterable